In [0]:
!pip install -q pyinterval

In [0]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import queue
import pydot
import json
from interval import interval, inf, imath
from networkx.drawing.nx_pydot import graphviz_layout
from collections import deque, OrderedDict
from google.colab import files
from sympy.solvers import solve
from sympy import Symbol
import math

In [0]:
def liniar_half(start, end, e):
  print('-----Iteration for [', start, ', ', end, ']')
  check = end - start
  if check <= e:
    print('FOUND X =', start, ' ', end)
    return
  X0 = interval([start, end])
  # function
  f = X0**3 - 6*X0**2 + 11*X0 - 6
  print('F: ', f)
  # if f == 0.0:
  #   liniar_half(start, mid, e)
  #   liniar_half(mid, end, e)
  #   return
  
  mid = (start+end)/2;
  #print('middle: ',  mid)
  if 0 in f:
    liniar_half(start, mid, e)
    liniar_half(mid, end, e)
    return
  
print("Лінійний алгоритм навпіл");
print("x^3 - 6*x^2 + 11*x - 6 = 0");
print("1, 2, 3");
print("[0; 6]");
print("------------");
x1 = 1;
x2 = 2;
x3 = 3;
e = 10**(-2)

a = 0;
b = 6;

liniar_half(a, b, e)

In [0]:
def liniar_mura(start, end, e):
  print('-----Iteration for [', start, ', ', end, '] ----------')
  
  X0 = interval([start, end])
  # function
  F = X0*X0*X0 - 6*X0*X0 + 11*X0 - 6
  if 0 not in F:
    print("0 not in F")
    return
  check = end - start
  print('width: ', check)
  if check < e:
    print('FOUND X =', start, ' ', end)
    return
  mid = (start+end)/2;
  f = mid**3 - 6*mid**2 + 11*mid - 6
  if f == 0.0:
    liniar_mura(start, mid, e)
    liniar_mura(mid, end, e)
    return
  print('function value:', f);
  print('middle: ',  end-start)
  # derivative
  f_derivative = 3*X0*X0 - 12*X0 + 11
  
  # print('function deritive: ', f_derivative);

  U = mid - f/f_derivative
  # print('U: ', U);
  
  X1 = U & X0
  [liniar_mura(x.inf, x.sup, e) for x in X1]
  
print("Лінійний алгоритм Мура");
print("x^3 - 6*x^2 + 11*x - 6 = 0");
print("1, 2, 3");
print("[0; 6]");
print("------------");
x1 = 1;
x2 = 2;
x3 = 3;
e = 10**(-6)

a = 0;
b = 6;

liniar_mura(a, b, e)

In [0]:
def liniar_hansen(start, end, e):
  print('-----Iteration for [', start, ', ', end, '] ----------')
  check = end - start
  print('width: ', check)
  X0 = interval([start, end])
  F = X0*X0*X0 - 6*X0*X0 + 11*X0 - 6
  if 0 not in F:
    print("0 not in F")
    return
  
  if check < e:
    print('FOUND X =', start, ' ', end)
    return;
  
  mid = (start+end)/2;
  print('middle: ',  mid)
  # function
  #f = mid*mid + mid - 2
  #f_derivative = 2*mid + 1
  f = mid**3 - 6*mid**2 + 11*mid - 6
  #fX0 = X0*X0*X0 - 6*X0*X0 + 11*X0 - 6
  
  print('function value:', f);
  # derivative
  F_derivative = 3*X0*X0 - 12*X0 + 11
  #F_derivative = 2 * X0 + 1
    
  temp = [x for x in F_derivative]
  # print("Temp: ", temp)
  c = temp[0].inf
  d = temp[0].sup
  print("c: ", c)
  print("d: ", d)
  
  if 0 not in F_derivative:
    print('Stop. F_derivative does not have 0')
    print("X0: ", X0)
    liniar_mura(start, end, e);
    return;

  
  if f == 0.0:
    print('FOUND X =', start, ' ', end)
    liniar_hansen(start, mid, e)
    liniar_hansen(mid, end, e)
    return
  elif f > 0.0:
    if c == 0.0:
      q = mid - f/d
      U = interval([-math.inf, q])
    elif d == 0.0:
      p = mid - f/c
      U = interval([p, math.inf])
    elif c < 0.0 and 0.0 < d:
      q = mid - f/d
      p = mid - f/c
      U = interval([-math.inf, q]) | interval([p, math.inf])
  elif f < 0.0:
    if c == 0.0:
      q = mid - f/d
      U = interval([q, math.inf])
    elif d == 0.0:
      p = mid - f/c
      U = interval([-math.inf, p])
    elif c < 0.0 and 0.0 < d:
      q = mid - f/d
      p = mid - f/c
      U = interval([-math.inf, p]) | interval([q, math.inf])
        
  X1 = U & X0
  
  print('X0: ', X0);
  print('U: ', U);
  print('X1: ', X1);

  result = list(map(lambda x: liniar_hansen(x.inf, x.sup, e), X1))
  # print("result: ", result)
  return


print("Лінійний алгоритм Хансена");
print("x^3 - 6*x^2 + 11*x - 6 = 0");
print("1, 2, 3");
print("[0; 4]");
print("------------");
x1 = 1;
x2 = 2;
x3 = 3;
e = 10**(-2)

a = 2;
b = 3;

liniar_hansen(a, b, e)

In [0]:
def liniar_krafchyk(start, end, e):
  print('-----Iteration for [', start, ', ', end, '] ----------')
  global i
  global i_dividebytwo
  i = i + 1
  
  X0 = interval([start, end])
  mid = (start+end)/2;
  
  f = mid**3 - 3*mid + 2
  print('f: ', f)
  print('width: ', end - start)

  fX0 = X0*X0*X0 - 3*X0 + 2
  if 0 not in fX0:
    print("0 not in F")
    return
  
  f_derivative = 3*mid**2 - 3
  F_derivative = 3*X0*X0 - 3
  
  check = end - start
  if check < e:
    print('FOUND X =', start, ' ', end)
    return
  
  if 0 in F_derivative:
    print('0 in F_derivative')
    if mid - start > e:
      liniar_krafchyk(start, mid, e)
      liniar_krafchyk(mid, end, e)
      i_dividebytwo = i_dividebytwo + 1
    return
      
  U = mid - f/f_derivative + (1 - (F_derivative)/f_derivative)*(X0-mid)
  
  X1 = U & X0
  
  print('X0: ', X0);
  print('U: ', U);
  print('X1: ', X1);
  
  if X0 == X1:
    #print('X0 == X1')
    liniar_krafchyk(start, mid, e)
    liniar_krafchyk(mid, end, e)
    i_dividebytwo = i_dividebytwo + 1
    return
  
  [liniar_krafchyk(x.inf, x.sup, e) for x in X1]
  
  
print("Лінійний алгоритм Крафчика");
print("x^3 - 6*x^2 + 11*x - 6 = 0");
print("1, 2, 3");
x1 = 1;
x2 = 2;
x3 = 3;
e = 10**(-6)

a = -2.5;
b = -1.25;

print('[', a, ';', b, ']');
print("------------");

i = 0
i_dividebytwo = 0
  
liniar_krafchyk(a, b, e)

print("------------");
print('Iterations: ', i)
print('Iterations (divide by two): ', i_dividebytwo)

Лінійний алгоритм Крафчика
x^3 - 6*x^2 + 11*x - 6 = 0
1, 2, 3
[ -2.5 ; -1.25 ]
------------
-----Iteration for [ -2.5 ,  -1.25 ] ----------
f:  1.033203125
width:  1.25
X0:  interval([-2.5, -1.25])
U:  interval([-2.691252587991719, -1.332556935817805])
X1:  interval([-2.5, -1.332556935817805])
-----Iteration for [ -2.5 ,  -1.332556935817805 ] ----------
f:  0.7120248481166875
width:  1.167443064182195
X0:  interval([-2.5, -1.332556935817805])
U:  interval([-2.5682333824817265, -1.4419662750549564])
X1:  interval([-2.5, -1.4419662750549564])
-----Iteration for [ -2.5 ,  -1.4419662750549564 ] ----------
f:  0.2561243239749622
width:  1.0580337249450436
X0:  interval([-2.5, -1.4419662750549564])
U:  interval([-2.434318805361475, -1.5668373812912868])
X1:  interval([-2.434318805361475, -1.5668373812912868])
-----Iteration for [ -2.434318805361475 ,  -1.5668373812912868 ] ----------
f:  -0.005204845281985726
width:  0.8674814240701882
X0:  interval([-2.434318805361475, -1.5668373812912868])